In [1]:
from __future__ import absolute_import
from google.cloud import language
import google.cloud
import pandas as pd
import numpy as np
import tqdm
import operator
import facebook
import json
import tweepy
import time
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import datetime


#Create client instance for Google Natural Language API
client = language.Client()

In [2]:
fb_comments_df = pd.DataFrame.from_csv('/Users/thays/Desktop/RDU_pitch/RDUpitch_all_fb_comments.csv', encoding='utf-8')
fb_comments_df.reset_index(inplace=True, drop = True)
fb_comments_df

,reply_comment_date,reply_comment_time,reply_comment_text,reply_comment_id,reply_username,reply_user_id,brand,post_date,post_time,post_text,post_id
0,2017-03-07,2017-03-07 00:23:13,Yeah! The 777 is back.... thank goodness !!!! ...,1581432665220060_1582700998426560,Jill Palombaro,1107902295988536,RDU,2017-03-05,22:33:42,The @americanair B-777 is now year round to #L...,145265145503493_1581432665220060
1,2017-03-06,2017-03-07 02:25:36,Hey Raleigh-Durham International Airport (RDU)...,1581432665220060_1581593685203958,Christopher Pippen,10155057211954547,RDU,2017-03-05,22:33:42,The @americanair B-777 is now year round to #L...,145265145503493_1581432665220060
2,2017-03-06,2017-03-07 00:17:00,Congratulations RDU!!!,1581432665220060_1581509498545710,Charlotte/Douglas International Airport Overlook,120221124739914,RDU,2017-03-05,22:33:42,The @americanair B-777 is now year round to #L...,145265145503493_1581432665220060
3,2017-03-06,2017-03-07 12:22:12,Almost anything would be better than the one t...,1581432665220060_1581990128497647,Emily Wilkins,10212033720176180,RDU,2017-03-05,22:33:42,The @americanair B-777 is now year round to #L...,145265145503493_1581432665220060
4,2017-03-06,2017-03-07 01:12:45,I have already bought our tickets for non stop...,1581432665220060_1581545908542069,Burak Eryigit,10155055452526505,RDU,2017-03-05,22:33:42,The @americanair B-777 is now year round to #L...,145265145503493_1581432665220060
5,2017-03-05,2017-03-07 23:45:31,Finally! I've been avoiding the RDU-LHR flight...,1581432665220060_1581489528547707,Rich Murray,10155217395928755,RDU,2017-03-05,22:33:42,The @americanair B-777 is now year round to #L...,145265145503493_1581432665220060
6,2017-03-05,2017-03-07 22:57:18,Now all we need is Virgin Atlantic RDU to LHR ...,1581432665220060_1581451508551509,Caron Beesley,10155389322039341,RDU,2017-03-05,22:33:42,The @americanair B-777 is now year round to #L...,145265145503493_1581432665220060
7,2017-03-06,2017-03-07 02:57:12,Thank you America air lines delta air line fro...,1581432665220060_1581612748535385,Farid Muti,10211646672699909,RDU,2017-03-05,22:33:42,The @americanair B-777 is now year round to #L...,145265145503493_1581432665220060
8,2017-03-06,2017-03-07 00:58:49,I heard AA was going to fly an MD-80 on this r...,1581432665220060_1581537178542942,Robert O Smith,10210899061464140,RDU,2017-03-05,22:33:42,The @americanair B-777 is now year round to #L...,145265145503493_1581432665220060
9,2017-03-06,2017-03-07 00:41:52,Don't expect the 777 to last with loads as hor...,1581432665220060_1581527755210551,Chris Murray,10104808865392122,RDU,2017-03-05,22:33:42,The @americanair B-777 is now year round to #L...,145265145503493_1581432665220060


In [3]:
# Sentiment Analysis of Replies

reply_sentiments_list = []
reply_magnitudes_list = []
reply_entities_name_list = []
reply_entities_salience_list = []
post_sentiments_list = []
post_magnitudes_list = []
post_entities_name_list = []
post_entities_salience_list = []

comments_list = list(zip(fb_comments_df['reply_comment_text'], fb_comments_df['post_text']))

for comment_iter in tqdm.tqdm(comments_list):
    
    comment = comment_iter[0]
    post = comment_iter[1]

    # Comment sentiment analysis
    try:
        reply_document = client.document_from_text(comment)
        reply_sentiment = reply_document.analyze_sentiment()
        reply_entities = reply_document.analyze_entities()
        reply_sentiment_score = reply_sentiment.score
        reply_magnitude = reply_sentiment.magnitude
        reply_entities_name = [str(x.name) for x in reply_entities[:3]]
        reply_entities_salience = [x.salience for x in reply_entities[:3]]
    except UnicodeEncodeError:
        sentiment_score = None
        magnitude = [None]
        entities_name = [None]
        entities_salience = [None]

    # Post sentiment analysis
    try:
        post_document = client.document_from_text(post)
        post_sentiment = post_document.analyze_sentiment()
        post_entities = post_document.analyze_entities()
        post_sentiment_score = post_sentiment.score
        post_magnitude = post_sentiment.magnitude
        post_entities_name = [str(x.name) for x in post_entities[:3]]
        post_entities_salience = [x.salience for x in post_entities[:3]]
    except UnicodeEncodeError:
        post_sentiment_score = None
        post_magnitude = [None]
        post_entities_name = [None]
        post_entities_salience = [None]


    reply_sentiments_list.append(reply_sentiment_score)
    reply_magnitudes_list.append(reply_magnitude)
    reply_entities_name_list.append(reply_entities_name)
    reply_entities_salience_list.append(reply_entities_salience)

    post_sentiments_list.append(post_sentiment_score)
    post_magnitudes_list.append(post_magnitude)
    post_entities_name_list.append(post_entities_name)
    post_entities_salience_list.append(post_entities_salience)
    
fb_comments_df['reply_sentiment'] = reply_sentiments_list
fb_comments_df['reply_magnitude'] = reply_magnitudes_list
fb_comments_df['reply_entities'] = reply_entities_name_list
fb_comments_df['reply_entity_salience'] = reply_entities_salience_list

fb_comments_df['post_sentiment'] = post_sentiments_list
fb_comments_df['post_magnitude'] = post_magnitudes_list
fb_comments_df['post_entities_name'] = post_entities_name_list
fb_comments_df['post_entity_salience'] = post_entities_salience_list

100%|██████████| 26/26 [00:14<00:00,  1.80it/s]


In [4]:
fb_comments_df.to_csv('/Users/thays/Desktop/RDU_pitch/RDUpitch_all_fb_comments_sentiment.csv', encoding='utf-8')